# YT Comments analysis

In [ ]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from datetime import datetime, timedelta, time
import pandas as pd
import json
import re
from api import API_KEY

channel_id = "UCWeg2Pkate69NFdBeuRFTAw" #Squeezie channel
etoiles = 'UCABf02qOye7XYapcK1M45LQ'
youtube = build('youtube', 'v3', developerKey=API_KEY)
exemple_video = "qCKyRhkhqoQ"
otp_recap = 'F7A8OCdmZ90'

### Class request
Class to handle youtube request since youtube api doesn't provide a request object

In [ ]:
class Request:
    """ Class Request handling youtube request as an object """
    def __init__(self, requestType,part=None, id=None, chart=None, regionCode=None, maxResults=None, pageToken=None, videoId=None):
        self.requestType = requestType
        self.part = part
        self.id = id
        self.chart = chart
        self.regionCode = regionCode
        self.maxResults = maxResults
        self.pageToken = pageToken
        self.videoId = videoId
        
    def execute(self):
        param = vars(self) # Fetch class attributes
        param = {x:y for x,y in list(param.items())[1:] if y} # Delete requestType ([1:]) and None attributes
        
        request = self.requestType.list(**param)
        return request.execute()

### Decorator
Decorator to retry when youtube request fails (mostly due to timeout erros)

In [ ]:
def retry_on_exception(max_attempts=5):
    def decorator(func):
        def wrapper(*args, **kwargs):
            attempts = 0
            while attempts < max_attempts:
                try:
                    result = func(*args, **kwargs)
                except Exception as e:
                    attempts += 1
                    if attempts == max_attempts:
                        return pd.DataFrame()
                        raise  # Relancer l'exception si le nombre maximal de tentatives est atteint
                    else:
                        print(f"{attempts}: Une exception s'est produite : {e}")
                else:
                    return result  # Retourner le résultat si aucune exception n'est levée
                # time.sleep(0.5)
        return wrapper
    return decorator

### Datetime convertions
Functions to convert iso formated date found in youtube api responses in datetime objects.

In [ ]:
def iso_toDatetime(iso_date:str):
    """Converts an ISO 8601 formatted date to a datetime object."""
    return datetime.strptime(iso_date[:-1], '%Y-%m-%dT%H:%M:%S')

def datetime_toISO(dt_obj:datetime):
    """Converts a datetime object to an ISO 8601 formatted date."""
    return dt_obj.isoformat()[:-7]  # remove microseconds

def iso_toDelta(iso_duration:str):
    """Converts an ISO 8601 formatted duration to a timedelta object."""
    match = re.match(r'PT(\d+D)*(\d+H)*(\d+M)*(\d+S)', iso_duration)
    days, hours, minutes, seconds = [int(x[:-1]) if x else 0 for x in match.groups()]
    return timedelta(days=days,hours=hours, minutes=minutes, seconds=seconds)

def delta_toISO(delta_obj:timedelta):
    """Converts a timedelta object to an ISO 8601 formatted duration."""
    hours = delta_obj.seconds // 3600
    minutes = (delta_obj.seconds % 3600) // 60
    seconds = delta_obj.seconds % 60
    
    daysStr = f"{delta_obj.days}D" if delta_obj.days != 0 else ""
    hoursStr = f"{hours}H" if hours != 0 else ""
    minutesStr = f"{minutes}M" if minutes != 0 else ""
    secondsStr = f"{seconds}S" if seconds != 0 else ""
    return f"PT{daysStr}{hoursStr}{minutesStr}{secondsStr}"

# print(iso_toDelta('PT4D3H20M9S'))
# print(delta_toISO(iso_toDelta('PT20M9S')))

### Fectching functions
Functions to fetch channels, comments ant top vids infos.

In [ ]:
def format_channel_dataDF(channel_data: dict) -> pd.DataFrame:
    """ Structure raw channel data """
    data = {
        "channel_name": [channel_data.get('snippet', {}).get('title')],
        "channel_id": [channel_data.get('id')],
        "country": [channel_data.get('snippet', {}).get('country',"")],
        **{k:[int(v)] for k,v in channel_data.get('statistics', {}).items() if k != "hiddenSubscriberCount"},
        "topics": [[wikilink.split('/')[-1] for wikilink in channel_data.get('topicDetails', {}).get('topicCategories', [])]],
    }
    return pd.DataFrame.from_dict(data)

In [ ]:
def get_channel_data(youtube, channel_id:str) -> dict[str|dict]:
    """ Request (by id) for most important channel stats """
    request = Request(
        requestType=youtube.channels(),
        part="snippet,contentDetails,statistics,topicDetails",
        id=channel_id
    )
    response = request.execute()
    rawData = response.get('items', [])[0]
    return format_channel_dataDF(rawData)
    

get_channel_data(youtube, etoiles)

In [ ]:
def format_video_dataDF(video_data: dict) -> pd.DataFrame:
    """ Structure raw video data """
    data = {
            "title": [video_data.get('snippet', {}).get('title')],
            "id": [video_data.get('id')],
            "publishedAt": [video_data.get('snippet', {}).get('publishedAt')],
            "duration" : [video_data.get('contentDetails').get('duration')],
            "ViewCount" : [int(video_data.get('statistics', {}).get('viewCount', 0))],
            "likeCount" : [int(video_data.get('statistics', {}).get('likeCount', 0))],
            "commentCount" : [int(video_data.get('statistics', {}).get('commentCount', 0))],  
            "tags" : [video_data.get('snippet', {}).get('tags')]
    }
    
    return pd.DataFrame.from_dict(data)

In [ ]:
def get_video_data(youtube, video_Id:str) -> dict[str|dict]:
    """ Request (by id) for most important video stats """
    request = Request(
        requestType=youtube.videos(),
        part="snippet,contentDetails,statistics,topicDetails",
        id=video_Id,
    )
    response = request.execute()
    
    rawData = response.get('items', [])[0]
    return format_video_dataDF(rawData)

get_video_data(youtube, exemple_video)

In [ ]:
def get_Most_Popular_Video(youtube, region:str) -> pd.DataFrame:
    """ Request for most populars videos stats """
    request = Request(
        requestType=youtube.videos(),
        part="snippet,contentDetails,statistics,topicDetails",
        chart="mostPopular",
        regionCode=region,
        maxResults=100,
        pageToken=''
    )
    response = request.execute()
    
    pages = [response]
    while response.get('nextPageToken'):
        request.pageToken = response.get('nextPageToken')
        response = request.execute()
        pages.append(response)
    
    top_videos = pd.concat([format_video_dataDF(videos) for page in pages for videos in page.get('items')]).reset_index(drop=True)
    top_videos['topID'] = top_videos.index + 1
    top_videos['region'] = region
    return top_videos

df:pd.DataFrame = get_Most_Popular_Video(youtube, 'FR')
df.sort_values(by=['fetchedDate'], ascending=False, inplace=True, kind='quicksort', ignore_index=True)
df

In [ ]:
def format_comment_data(comment:dict) -> dict[str|dict]:
    """ Structure raw comment data """
    data = {
        "id": comment.get('id'),
        "comment": comment.get('snippet', {}).get('textOriginal'),
        # "viewerRating": comment.get('snippet', {}).get('viewerRating'),
        "likeCount": comment.get('snippet', {}).get('likeCount'),
        "publishedAt": comment.get('snippet', {}).get('publishedAt'),
        "updatedAt": comment.get('snippet', {}).get('updatedAt')
        }
    
    return data

def format_threadedComment_data(comment:dict) -> dict[str|dict]:
    """ Structure raw threaded comment data """
    data = {
        **format_comment_data(comment.get('snippet', {}).get('topLevelComment')),
        "totalReplyCount": comment.get('snippet', {}).get('totalReplyCount'),
        # "replies": [format_comment_data(com) for com in comment.get('replies', {}).get('comments', [])]
        }
    
    return data

In [ ]:
def format_comment_dataDF(comment:dict) -> pd.DataFrame:
    """ Structure raw comment data """
    data = {
        "id": [comment.get('id')],
        "comment": [comment.get('snippet', {}).get('textOriginal')],
        # "viewerRating": comment.get('snippet', {}).get('viewerRating'),
        "likeCount": [int(comment.get('snippet', {}).get('likeCount'))],
        "publishedAt": [comment.get('snippet', {}).get('publishedAt')],
        "updatedAt": [comment.get('snippet', {}).get('updatedAt')]
        }
    
    return data

def format_threadedComment_dataDF(comment:dict) -> pd.DataFrame:
    """ Structure raw threaded comment data """
    data = {
        **format_comment_dataDF(comment.get('snippet', {}).get('topLevelComment')),
        "totalReplyCount": [int(comment.get('snippet', {}).get('totalReplyCount'))],
        # "replies": [format_comment_data(com) for com in comment.get('replies', {}).get('comments', [])]
        }
    
    return data

In [ ]:
def format_commentPageDF(page:list[dict]) -> pd.DataFrame:
    data = {}
    for comment in page:
        topLevelComment = comment.get('snippet', {}).get('topLevelComment')
        data.setdefault("id", []).append(topLevelComment.get('id')) 
        data.setdefault("comment", []).append(topLevelComment.get('snippet', {}).get('textOriginal'))
        data.setdefault("likeCount", []).append(int(topLevelComment.get('snippet', {}).get('likeCount')))
        data.setdefault("publishedAt", []).append(topLevelComment.get('snippet', {}).get('publishedAt'))
        data.setdefault("updatedAt", []).append(topLevelComment.get('snippet', {}).get('updatedAt'))
        data.setdefault("totalReplyCount", []).append(int(comment.get('snippet', {}).get('totalReplyCount')))
    return pd.DataFrame(data)

In [ ]:
def get_comment(youtube,comment_id:str) -> dict[str|dict]:
    """ Request (by id) for most important comment stats """
    request = Request(
        requestType=youtube.comments(),
        part="snippet,id",
        id=comment_id,
    )
    response = request.execute()
    rawData = response.get('items')[0]
    return pd.DataFrame(format_comment_dataDF(rawData))

get_comment(youtube, 'UgwUQR2JJFJSkihWLhx4AaABAg')

In [ ]:
@retry_on_exception(max_attempts=3)
def get_video_commentThreads(youtube, videoID:str, maxComments:int) -> dict[str|list]:
    """ Request (by id) for all comments of a videos """
    request = Request(
        requestType=youtube.commentThreads(),
        part="snippet,id,replies",
        videoId=videoID,
        maxResults=100
    )
    response = request.execute()
    
    maxComments -= response.get('pageInfo', {}).get('totalResults')
    comments = format_commentPageDF(response.get('items',{}))
    while response.get('nextPageToken'):
        request.pageToken = response.get('nextPageToken')
        response = request.execute()
        comments = pd.concat([comments, format_commentPageDF(response.get('items',{}))], ignore_index=True)
        if (maxComments:= maxComments - response.get('pageInfo', {}).get('totalResults')) <= 0:
            break
        
    comments['videoID'] = videoID
    comments['fetchedDate'] = datetime.today()
    return comments

get_video_commentThreads(youtube, 'XqZsoesa55w', 1000).reset_index().sort_values(by=['likeCount', 'totalReplyCount'], ascending=False, ignore_index=True)
# get_video_commentThreads(youtube, 'FkXhKu80CWU', 1000)

# Fetching Top Videos
The goal is to fetch the top 200 videos everyday and to get their comments a week after publishing.

In [ ]:
def json_toDF(topvidsFile):
    """ transfrom topVideos.json to dataframe then to .csv"""
    with open(topvidsFile, 'r') as f:
        data = json.load(f)
    data.pop('lastUpdate')
    videosList = {}
    for region,dateEntries in data.items():
        # print(region, type(dateEntries))
        for date, videos in dateEntries.items():
            for id, video in enumerate(videos):
                videosList.setdefault('title', []).append(video.get('title'))
                videosList.setdefault('id', []).append(video.get('id'))
                videosList.setdefault('publishedAt', []).append(video.get('publishedAt'))
                videosList.setdefault('duration', []).append(video.get('duration'))
                videosList.setdefault('ViewCount', []).append(video.get('ViewCount'))
                videosList.setdefault('likeCount', []).append(video.get('likeCount'))
                videosList.setdefault('commentCount', []).append(video.get('commentCount'))
                videosList.setdefault('tags', []).append(video.get('tags'))
                videosList.setdefault('fetchedDate', []).append(date) 
                videosList.setdefault('topID', []).append(id+1)
                videosList.setdefault('region', []).append(region)
    
    df = pd.DataFrame(videosList)
    df.to_csv('db/top.csv', index=False)
    # fetchedDate topID region
json_toDF('db/topVideos.json')

In [ ]:
def push_top_vids(topvidsFile:str, regions:list[str], minElapsedTime:int)-> None:
    """ Fetch top 200 vids per region (per <minElapsedTime>) and push in json <topvidsFile> """
    today = datetime.today()
    try:
        df = pd.read_csv(topvidsFile)
    except pd.errors.EmptyDataError:
        df = pd.DataFrame()
    
    if 'fetchedDate' in df:
        df['fetchedDate'] = pd.to_datetime(df['fetchedDate'], format='ISO8601')
        lastUpdate = df.sort_values(by=['fetchedDate'], ascending=False, ignore_index=True).loc[0]['fetchedDate']
        delta = today - lastUpdate
        if delta.total_seconds() // 3600 < minElapsedTime:
            raise Exception(f'The fetch request has be done too soon. Next request available in {24-(delta.total_seconds() // 3600)}h. Last update done : {lastUpdate}')

    # Fetching
    for reg in regions:
        top200 = get_Most_Popular_Video(youtube, reg)
        top200['fetchedDate'] = today
        top200['fetchedComments'] = False
        df = pd.concat([df, top200])
        
    df.to_csv('db/dailyTop200.csv', index=False) 

push_top_vids('db/dailyTop200.csv', ['FR', 'US'], 24)

In [ ]:
def fetch_topVids_comments(topvidsFile:str, minElapsedCommentsTime:int, maxComments:int = 1000) -> None:
    today = datetime.today()
    try:
        df = pd.read_csv(topvidsFile)
    except pd.errors.EmptyDataError:
        df = pd.DataFrame()
    df['publishedAt'] = pd.to_datetime(df['publishedAt'], format='ISO8601')
    
    print(df.dtypes)
    print(df.shape)
    
    uniqueID = df.drop_duplicates(['id'])
    uniqueID = uniqueID[uniqueID['fetchedComments'] == False]
    id_list = list(uniqueID['id'])
    
    comments = pd.DataFrame()
    for id in id_list:
        if (pd.to_datetime(today) - uniqueID.loc[uniqueID['id'] == id, 'publishedAt'].values[0]).days >= minElapsedCommentsTime:
            print(id)
            comments = pd.concat([comments, get_video_commentThreads(youtube, id, maxComments)])
            df.loc[df['id'] == id, 'fetchedComments'] = True
            
    return comments
        
    
fetch_topVids_comments('db/dailyTop200.csv', 7, 1000)

In [ ]:
df = pd.read_csv('db/dailyTop200.csv')
df

In [ ]:
def create_comment_queue(topvidsFile:str, commentsQueue:str) -> None:
    """ Create a json file <commentsQueue> that stores video IDs which comments have not been fetched """
    with open(topvidsFile, 'r') as f:
        data:dict = json.load(f)
    with open(commentsQueue, 'r') as f:
        queue:dict = json.load(f)   
        
    data.pop('lastUpdate', None)
    date_to_fetch = [dates for dates in list(list(data.values())[0].keys()) if dates not in queue.keys()]
    
    if date_to_fetch: # is empty
        for date in date_to_fetch:  
            comment_list = []
            for region in data.keys():
                # print(f"{region}: {date}. Size {len(data.get(region,{}).get(date,[]))}")
                for video in data.get(region,{}).get(date,[]):
                    comment_list += [
                        {'region': region, 
                        'dateEntry': date, 
                        'id': video.get('id'),
                        'publishedAt': video.get('publishedAt')
                        }
                    ]
            queue[date] = comment_list
        
        with open(commentsQueue, 'w') as fichier:
            json.dump(queue, fichier)    
    else:
        print("already in !")

In [ ]:

def fetch_topVids_comments(commentsQueue:str, commentList:str, minElapsedCommentsTime:int) -> None:
    """ Fetches vids comments from <commentsQueue> after minElapsedComments and output them in <commentList>"""
    today = datetime.today()
    with open(commentsQueue, 'r') as f:
        queue:dict[list] = json.load(f)
    with open(commentList, 'r') as f:
        comments:dict[list] = json.load(f)
    
    data={}
    remove_indices:dict[list] = {}
    for date,vids in queue.items():
        for i,video in enumerate(vids):
            if (today - iso_toDatetime(video.get('publishedAt'))).days >= minElapsedCommentsTime: 
                start = time.time()
                data[video.get('id')] = get_video_commentThreads(youtube, video.get('id'), 1000) # crashes a lot
                print(f"{i} in {time.time()-start}s",end='\n\n')
                if date not in remove_indices.keys():
                     remove_indices[date] = []
                remove_indices[date].append(i)
            else:print(f"{i} not yet",end='\n\n')
        
    queue = {date:[video for i, video in enumerate(videos) if i not in remove_indices[date]] for date,videos in queue.items()}  
     
    with open(commentsQueue, 'w') as fichier:
            json.dump(queue, fichier)  
               
    with open(commentList, 'w') as fichier:
            json.dump({**comments, **data}, fichier) # merging data

In [ ]:
REGION = ['FR', 'US']
topvids = 'db/topVideos.json'
commentsQueue="db/commentQueue.json"
commentList = "db/commentList.json"
minElapsedTime = 24 # Hours
minElapsedCommentsTime = 17 # days

push_top_vids(topvids, REGION, minElapsedTime)
    
# create_comment_queue(topvids, commentsQueue)
            
# fetch_topVids_comments(commentsQueue, commentList, minElapsedCommentsTime)    